In [ ]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
! pip install accelerate
# Ανδρεάδης Δημήτρης 03119403
# Αστρεινίδης Ζαφείρης 03119053

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference 
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
from datasets import load_dataset,interleave_datasets

# insert your code here
ds = load_dataset("yelp_polarity")
ds.shape

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'train': (560000, 2), 'test': (38000, 2)}

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
# insert your code here
train_dataset_0 = ds['train'].filter(lambda e: e['label']==0)
train_dataset_1 = ds['train'].filter(lambda e: e['label']==1)
train_dataset = interleave_datasets([train_dataset_0, train_dataset_1]).select(range(300)).shuffle()

eval_dataset_0 = ds['test'].filter(lambda e: e['label']==0)
eval_dataset_1 = ds['test'].filter(lambda e: e['label']==1)
eval_dataset = interleave_datasets([eval_dataset_0, eval_dataset_1]).select(range(300)).shuffle()
# The training dataset is ensured to have equal number of samples for the two labels, interleaved with each other

Filter:   0%|          | 0/560000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/560000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/38000 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset.unique('label'))
print(eval_dataset.unique('label'))

[0, 1]
[0, 1]


# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here
from transformers import AutoTokenizer, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
    #return tokenizer(examples["text"], truncation=True)

tokenized_train_dataset= train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset= eval_dataset.map(tokenize_function, batched=True)


# insert your code here

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
type(train_dataset)

datasets.arrow_dataset.Dataset

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή. 

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [ ]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16, learning_rate=0.0005, num_train_epochs=3, optim= "adamw_torch")


# insert your code here
# optimizer
# scheduler

# We use the adamw_torch optimizer and the default linear scheduler
# etc

In [ ]:
# The default learning rate(lr) is 5e-5
# batch_size   lr         accuracy
# 16           default    0.89  
# 32           default    0.78
# 8            0.0001     0.88
# 16           0.0001     0.916
# 32           0.0001     0.903
# 16           0.005      0.71
# 16           0.001      0.5

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.714318,0.500000
2,No log,0.695034,0.500000
3,No log,0.693143,0.500000


# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)
from datasets import load_dataset
ds = load_dataset('piqa')
ds

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Dataset piqa downloaded and prepared to /root/.cache/huggingface/datasets/piqa/plain_text/1.1.0/6c611c1a9bf220943c4174e117d3b660859665baf1d43156230116185312d011. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 16113
    })
    test: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 3084
    })
    validation: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 1838
    })
})

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
val_100 = ds['validation'].shuffle().select(range(100))
val_100

Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})

In [ ]:
# insert your code here (models)
from transformers import AutoTokenizer, AutoModelForMultipleChoice

tokenizer1 = AutoTokenizer.from_pretrained("bert-base-uncased")
model1 = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased") 

tokenizer2 = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model2 = AutoModelForMultipleChoice.from_pretrained("distilbert-base-uncased")

tokenizer3 = AutoTokenizer.from_pretrained("roberta-base")
model3 = AutoModelForMultipleChoice.from_pretrained("roberta-base")

tokenizer4 = AutoTokenizer.from_pretrained("albert-base-v2")
model4 = AutoModelForMultipleChoice.from_pretrained("albert-base-v2")

tokenizer5 = AutoTokenizer.from_pretrained("YituTech/conv-bert-base")
model5 = AutoModelForMultipleChoice.from_pretrained("YituTech/conv-bert-base")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForMultipleChoice: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultipleChoice: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [ ]:
# For each model tokenizer we define a tokenize_function that is applied to the preprocessed dataset and we take 5 different tokenized sets
def preprocess(example):
    example['first_sentence']=[example['goal']]*2
    example['second_sentence']=[example[f'sol{i}'] for i in [1,2]]
    return example

def tokenize_function1(examples):
    tokenized_examples=tokenizer1(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

def tokenize_function2(examples):
    tokenized_examples=tokenizer2(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

def tokenize_function3(examples):
    tokenized_examples=tokenizer3(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

def tokenize_function4(examples):
    tokenized_examples=tokenizer4(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

def tokenize_function5(examples):
    return(tokenizer5(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True))

In [ ]:
val_100_processed = val_100.map(preprocess)
tokenized1_val_100 = val_100_processed.map(tokenize_function1)
tokenized2_val_100 = val_100_processed.map(tokenize_function2)
tokenized3_val_100 = val_100_processed.map(tokenize_function3)
tokenized4_val_100 = val_100_processed.map(tokenize_function4)
tokenized5_val_100 = val_100_processed.map(tokenize_function5)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Each tokenizer uses a different tokenized set and the same compute metric with the others. By calling trainer.evaluate we do not have to define
# an accuracy computing function ourselves
trainer = Trainer(
    model=model1,
    eval_dataset=tokenized1_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.6919118762016296,
 'eval_accuracy': 0.52,
 'eval_runtime': 6.0414,
 'eval_samples_per_second': 16.553,
 'eval_steps_per_second': 2.152}

In [ ]:
trainer = Trainer(
    model=model2,
    eval_dataset=tokenized2_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.6931222677230835,
 'eval_accuracy': 0.58,
 'eval_runtime': 207.6293,
 'eval_samples_per_second': 0.482,
 'eval_steps_per_second': 0.063}

In [ ]:
trainer = Trainer(
    model=model3,
    eval_dataset=tokenized3_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.693092405796051,
 'eval_accuracy': 0.54,
 'eval_runtime': 429.7725,
 'eval_samples_per_second': 0.233,
 'eval_steps_per_second': 0.03}

In [ ]:
trainer = Trainer(
    model=model4,
    eval_dataset=tokenized4_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.6929224133491516,
 'eval_accuracy': 0.48,
 'eval_runtime': 512.2522,
 'eval_samples_per_second': 0.195,
 'eval_steps_per_second': 0.025}

In [ ]:
trainer = Trainer(
    model=model5,
    eval_dataset=tokenized5_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.6958904266357422,
 'eval_accuracy': 0.37,
 'eval_runtime': 435.5168,
 'eval_samples_per_second': 0.23,
 'eval_steps_per_second': 0.03}

## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model1 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
sentence_model2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sentence_model3 = SentenceTransformer('shibing624/text2vec-base-chinese')
sentence_model4 = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
sentence_model5 = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
sentence_model6 = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model1 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [ ]:
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = sentence_model1.encode(sentences)
print(get_cosine_similarity(embeddings[0], embeddings[1]))
embeddings = sentence_model2.encode(sentences)
print(get_cosine_similarity(embeddings[0], embeddings[1]))
embeddings = sentence_model3.encode(sentences)
print(get_cosine_similarity(embeddings[0], embeddings[1]))
embeddings = sentence_model4.encode(sentences)
print(get_cosine_similarity(embeddings[0], embeddings[1]))
embeddings = sentence_model5.encode(sentences)
print(get_cosine_similarity(embeddings[0], embeddings[1]))
embeddings = sentence_model6.encode(sentences)
print(get_cosine_similarity(embeddings[0], embeddings[1]))

0.40488476
0.40455928
0.7684448
0.61136675
0.40418404
0.57836944


Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή. 

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
# insert your code here (load dataset)
from datasets import load_dataset
ds = load_dataset('truthful_qa','generation')

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dsfiltered = ds['validation'].filter(lambda e: len(e['correct_answers'])>=2).shuffle().select(range(100))
# We ensure that each sample of the dataset has more than 2 correct answers

In [ ]:
dsfilteredwithlabel = dsfiltered.add_column('label', [0]*100)

Flattening the indices:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
dsfilteredwithlabel

Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source', 'label'],
    num_rows: 100
})

In [ ]:
# insert your code here (load models for semantic similarity and QA)
def preprocess4(example):
    example['first_sentence']=[example['question']]*4
    example['second_sentence']=[example['best_answer'],example['correct_answers'][0],example['correct_answers'][1],example['incorrect_answers'][0]]
    return example




In [ ]:
dsfilteredwithlabel_processed = dsfilteredwithlabel.map(preprocess4)

tokenized1_val_100 = dsfilteredwithlabel_processed.map(tokenize_function1)
tokenized2_val_100 = dsfilteredwithlabel_processed.map(tokenize_function2)
tokenized3_val_100 = dsfilteredwithlabel_processed.map(tokenize_function3)
tokenized4_val_100 = dsfilteredwithlabel_processed.map(tokenize_function4)
tokenized5_val_100 = dsfilteredwithlabel_processed.map(tokenize_function5)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import evaluate
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics2(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    optpredictions = optimal_predictions(predictions, sentence_model6)
    return metric.compute(predictions=optpredictions, references=labels)
# We define a second accuracy computing function for calculating the percentage of optimal correct answers

In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)
def optimal_predictions(predictions,model):
    for i in range(len(predictions)):
      if (predictions[i]==0):
        predictions[i]=0
      elif (predictions[i]==3):
        predictions[i]=1
      else :
        sentences = [dsfilteredwithlabel[i]['correct_answers'][predictions[i]-1], dsfilteredwithlabel[i]['best_answer']]
        embeddings = model.encode(sentences)
        if (get_cosine_similarity(embeddings[0], embeddings[1])>0.95):
          predictions[i]=0
        else:
          predictions[i]=1
    return predictions

# If the model predict a correct answer instead of a best answer, this counts a correct only if the correct answer has a sentence similarity
# of more than 0.95. So we try all models and all sentence models to this aim and we note all the corresponding model accuracies for optimal correctness

In [ ]:
from transformers import Trainer
metric = evaluate.load("accuracy")

trainer = Trainer(
    model=model5,
    eval_dataset=tokenized5_val_100,
    compute_metrics=compute_metrics2,
)
trainer.evaluate()

{'eval_loss': 1.3877208232879639,
 'eval_accuracy': 0.32,
 'eval_runtime': 1001.026,
 'eval_samples_per_second': 0.1,
 'eval_steps_per_second': 0.013}

In [ ]:
# model                        sentence_model                                      accuracy
# bert-base-uncased            sentence-transformers/all-mpnet-base-v2             0.32
# bert-base-uncased            sentence-transformers/all-MiniLM-L6-v2              0.31
# bert-base-uncased            shibing624/text2vec-base-chinese                    0.31
# bert-base-uncased            sentence-transformers/multi-qa-mpnet-base-dot-v1    0.31
# bert-base-uncased            sentence-transformers/all-MiniLM-L12-v2             0.32
# bert-base-uncased            sentence-transformers/bert-base-nli-mean-tokens     0.32
# distilbert-base-uncased      sentence-transformers/all-mpnet-base-v2             0.29
# distilbert-base-uncased      sentence-transformers/all-MiniLM-L6-v2              0.28
# distilbert-base-uncased      shibing624/text2vec-base-chinese                    0.27
# distilbert-base-uncased      sentence-transformers/multi-qa-mpnet-base-dot-v1    0.28
# distilbert-base-uncased      sentence-transformers/all-MiniLM-L12-v2             0.28
# distilbert-base-uncased      sentence-transformers/bert-base-nli-mean-tokens     0.3
# roberta-base                 sentence-transformers/all-mpnet-base-v2             0.46
# roberta-base                 sentence-transformers/all-MiniLM-L6-v2              0.46
# roberta-base                 shibing624/text2vec-base-chinese                    0.47
# roberta-base                 sentence-transformers/multi-qa-mpnet-base-dot-v1    0.46
# roberta-base                 sentence-transformers/all-MiniLM-L12-v2             0.46
# roberta-base                 sentence-transformers/bert-base-nli-mean-tokens     0.47
# albert-base-v2               sentence-transformers/all-mpnet-base-v2             0.4
# albert-base-v2               sentence-transformers/all-MiniLM-L6-v2              0.38
# albert-base-v2               shibing624/text2vec-base-chinese                    0.37
# albert-base-v2               sentence-transformers/multi-qa-mpnet-base-dot-v1    0.39
# albert-base-v2               sentence-transformers/all-MiniLM-L12-v2             0.38
# albert-base-v2               sentence-transformers/bert-base-nli-mean-tokens     0.38
# YituTech/conv-bert-base      sentence-transformers/all-mpnet-base-v2             0.3
# YituTech/conv-bert-base      sentence-transformers/all-MiniLM-L6-v2              0.3      
# YituTech/conv-bert-base      shibing624/text2vec-base-chinese                    0.3
# YituTech/conv-bert-base      sentence-transformers/multi-qa-mpnet-base-dot-v1    0.31
# YituTech/conv-bert-base      sentence-transformers/all-MiniLM-L12-v2             0.3
# YituTech/conv-bert-base      sentence-transformers/bert-base-nli-mean-tokens     0.32



## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση. 

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
# insert your code here (load dataset)
from datasets import load_dataset
ds = load_dataset('winogrande','winogrande_debiased')
ds

Generating train split:   0%|          | 0/9248 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

Dataset winogrande downloaded and prepared to /root/.cache/huggingface/datasets/winogrande/winogrande_debiased/1.1.0/a826c3d3506aefe0e9e9390dcb53271070536586bab95849876b2c1743df56e2. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 9248
    })
    test: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 1767
    })
    validation: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 1267
    })
})

In [ ]:
val_100 = ds['validation'].shuffle(seed=42).select(range(100))

In [ ]:
val_100[1]

{'sentence': 'The clothing in the north was warmer than the clothing in the south because there was more snow in the _ .',
 'option1': 'south',
 'option2': 'north',
 'answer': '2'}

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [ ]:
# insert your code here (load models)
from transformers import pipeline

# insert your code here (create pipelines)
classifier1 = pipeline(model="DeepPavlov/roberta-large-winogrande")
classifier2 = pipeline(model="sileod/deberta-v3-base-tasksource-nli")
classifier3 = pipeline(model="sileod/deberta-v3-large-tasksource-nli")

# We load the models with pipeline tool this time

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
# insert your code here (function for predicting best fill)
# classifier1
acc=0
for i in range(100):
  sentence_1,sentence_2 = str(val_100[i]['sentence']).split('_')
  sentence_2a = str(val_100[i]['option1']) + sentence_2
  sentence_2b = str(val_100[i]['option2']) + sentence_2
  sentence_1,sentence_2 = sentence_1 + sentence_2a, sentence_1 + sentence_2b
  class1,class2 = classifier1(sentence_1)[0], classifier1(sentence_2)[0]
  label1,label2 = class1['label'], class2['label']
  score1,score2 = class1['score'], class2['score']
  if (label1 != label2):
    if (label1=='True'):
      if (val_100[i]['answer']) == '1':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '2':
        acc=acc+1
  elif(label1=='True'):
    if (score1>score2):
      if (val_100[i]['answer']) == '1':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '2':
        acc=acc+1
  else:
    if (score1>score2):
      if (val_100[i]['answer']) == '2':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '1':
        acc=acc+1
  
print(acc/100.0)
# We define an accuracy calculating function for each of the 3 models. This is necessary as each model returns different labels for classifying a training dataset
# and we can infer from these labels the correct label for our specific problem. As there is not a pipeline for multiple choice task, this task was reduced to 
# sequence classification for the first two models and zero-shot classification for the third (the labels in the third model are inserted by the user)

0.68


In [ ]:
# insert your code here (function for predicting best fill)
# classifier2
acc=0
for i in range(100):
  sentence_1,sentence_2 = str(val_100[i]['sentence']).split('_')
  sentence_2a = str(val_100[i]['option1']) + sentence_2
  sentence_2b = str(val_100[i]['option2']) + sentence_2
  sentence_1,sentence_2 = sentence_1 + sentence_2a, sentence_1 + sentence_2b
  class1,class2 = classifier2(sentence_1,return_all_scores = True)[0], classifier2(sentence_2,return_all_scores = True)[0]
  for i in range(3):
    if(class1[i]['label']=='entailment'):
      score1= class1[i]['score']
    if(class2[i]['label']=='entailment'):
      score2= class2[i]['score']

  if (score1>score2 and val_100[i]['answer'] == '1'):
    acc=acc+1
  if (score1<score2 and val_100[i]['answer'] == '2'):
    acc=acc+1
  
print(acc/100.0)

0.54


In [ ]:
# insert your code here (function for predicting best fill)
# classifier3
acc=0
for i in range(100):
  sentence_1,sentence_2 = str(val_100[i]['sentence']).split('_')
  sentence_2a = str(val_100[i]['option1']) + sentence_2
  sentence_2b = str(val_100[i]['option2']) + sentence_2
  sentence_1,sentence_2 = sentence_1 + sentence_2a, sentence_1 + sentence_2b
  class1,class2 = classifier3(sentence_1,['True','False']), classifier3(sentence_2,['True','False'])
  label1,label2 = class1['labels'][0], class2['labels'][0]
  score1,score2 = class1['scores'][0], class2['scores'][0]
  if (label1 != label2):
    if (label1=='True'):
      if (val_100[i]['answer']) == '1':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '2':
        acc=acc+1
  elif(label1=='True'):
    if (score1>score2):
      if (val_100[i]['answer']) == '1':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '2':
        acc=acc+1
  else:
    if (score1>score2):
      if (val_100[i]['answer']) == '2':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '1':
        acc=acc+1
  
print(acc/100.0)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0.84
